## 합성곱/풀링 계층 구현

In [3]:
from common.util import im2col
import numpy as np

In [4]:
x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1, 5,5,stride=1, pad=0)
print(col1.shape)

(9, 75)


In [6]:
x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2, 5,5,stride=1, pad=0)
print(col2.shape)

(90, 75)


### 합성곱

In [7]:
class Convolution:
    def __init__(self, W, b, stride=1, pad=0):
        self.W = W
        self.b = b
        self.stride= stride
        self.pad= pad
        
    def forward(self, x):
        FN,C,FH,FW = self.W.shape
        N,C,H,W = x.shape
        out_h = int(1+(H+2*self.pad-FH) / self.stride)
        out_w = int(1+(W+2*self.pad-FW) / self.stride)
        
        col = im2col(x,FH,FW, self.stride, self.pad)
        col_W = self.W.reshape(FN,-1).T # filter 전개
        out = np.dot(col,col_W)+self.b
        
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)
        
        return out

### 풀링

In [8]:
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N,C,H,W = x.shape
        out_h = int(1+(H-self.pool_h) / self.stride)
        out_w = int(1+(W-self.pool_w) / self.stride)
        
        # 전개(1)
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        # 최대값(2)
        out = np.max(col, axis=1)
        
        # 성형(3)
        out = out.reshape(N, out_h, out_w, C).transpose(0,3,1,2)
        
        return out

## CNN 구현

In [41]:
from collections import OrderedDict

class SimpleConvNet:
    def __init__(self, input_dim=(1,28,28), conv_param={'filter_num':30, 'filter_size' : 5, 'pad' : 0, 'stride':1}, hidden_size=100, output_size=10, weight_init_std=0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / filter_stride+1
        pool_output_size = int(filter_num * (conv_output_size/2) * (conv_output_size/2))
        
        # 가중치 변수 초기화
        self.params={}
        self.params['W1'] = weight_init_std * np.random.randn(filter_num, input_dim[0], filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * np.random.randn(pool_output_size, hidden_size)
        self.params['b2'] = np.zeros(hidden_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        # CNN 구성 계층 생성(Relu, Pool, Affine)
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'], conv_param['stride'], conv_param['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2, stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
        
    ## predict
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    ## 손실함수
    def loss(self, x,t):
        y = self.predict(x)
        return self.last_layer.forward(y,t)
    
    def accuracy(self, x, t, batch_size=100):
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        acc = 0.0
        
        for i in range(int(x.shape[0] / batch_size)):
            tx = x[i*batch_size:(i+1)*batch_size]
            tt = t[i*batch_size:(i+1)*batch_size]
            y = self.predict(tx)
            y = np.argmax(y, axis=1)
            acc += np.sum(y == tt) 
        
        return acc / x.shape[0]

    def numerical_gradient(self, x, t):
        """기울기를 구한다（수치미분）.

        Parameters
        ----------
        x : 입력 데이터
        t : 정답 레이블

        Returns
        -------
        각 층의 기울기를 담은 사전(dictionary) 변수
            grads['W1']、grads['W2']、... 각 층의 가중치
            grads['b1']、grads['b2']、... 각 층의 편향
        """
        loss_w = lambda w: self.loss(x, t)

        grads = {}
        for idx in (1, 2, 3):
            grads['W' + str(idx)] = numerical_gradient(loss_w, self.params['W' + str(idx)])
            grads['b' + str(idx)] = numerical_gradient(loss_w, self.params['b' + str(idx)])

        return grads
    
    ## 오차역전파
    def gradient(self,x,t):
        # 순전파
        self.loss(x,t)
        
        # 역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
            
        # 결과저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads
        

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mnist import load_mnist
from common.trainer import Trainer
from common.layers import *
from common.gradient import numerical_gradient
import pickle

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=False)

# 시간이 오래 걸릴 경우 데이터를 줄인다.
#x_train, t_train = x_train[:5000], t_train[:5000]
#x_test, t_test = x_test[:1000], t_test[:1000]

max_epochs = 20

network = SimpleConvNet(input_dim=(1,28,28), 
                        conv_param = {'filter_num': 30, 'filter_size': 5, 'pad': 0, 'stride': 1},
                        hidden_size=100, output_size=10, weight_init_std=0.01)
                        
trainer = Trainer(network, x_train, t_train, x_test, t_test,
                  epochs=max_epochs, mini_batch_size=100,
                  optimizer='Adam', optimizer_param={'lr': 0.001},
                  evaluate_sample_num_per_epoch=1000)
trainer.train()

# 매개변수 보존
network.save_params("params.pkl")
print("Saved Network Parameters!")

# 그래프 그리기
markers = {'train': 'o', 'test': 's'}
x = np.arange(max_epochs)
plt.plot(x, trainer.train_acc_list, marker='o', label='train', markevery=2)
plt.plot(x, trainer.test_acc_list, marker='s', label='test', markevery=2)
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

train loss:2.29976848525
=== epoch:1, train acc:0.153, test acc:0.121 ===
train loss:2.29703865909
train loss:2.29349496043
train loss:2.28454110069
train loss:2.28090466449
train loss:2.27521645219
train loss:2.27057422924
train loss:2.23266515159
train loss:2.23823527863
train loss:2.20282523352
train loss:2.17321902931
train loss:2.11208674102
train loss:2.09596382843
train loss:2.08313657431
train loss:2.04480220811
train loss:1.9026648648
train loss:1.90757316162
train loss:1.80500385439
train loss:1.81023048658
train loss:1.6544570693
train loss:1.65694884909
train loss:1.48374499791
train loss:1.37660894104
train loss:1.49649361073
train loss:1.3093224393
train loss:1.3077817065
train loss:0.97312500412
train loss:1.05683628265
train loss:0.917699596484
train loss:0.883573149939
train loss:0.842165926696
train loss:0.872841597865
train loss:0.920154413441
train loss:0.966854552735
train loss:0.692182010461
train loss:0.801175067131
train loss:0.726123106729
train loss:0.73624652